## ABM Model 

## Imports

In [ ]:
import numpy as np 
import random 
import pandas as pd
from demand import ElasticDemand
from battery import InformedTraderBattery, OptimalPolicyBattery
from supply import Supply 
from amm import AMM
from model import Model
import os

%load_ext autoreload
%autoreload 2


### Solar Generation Model

In [ ]:
def generate_daily_solar_profile(sunrise=6, sunset=20, mean_pmax=15, std_pmax=1, seed=None):
    """
    Generate a 24-hour solar irradiance profile using a cosine model for daylight hours.
    
    Parameters:
    - sunrise: Hour of sunrise (e.g., 6)
    - sunset: Hour of sunset (e.g., 18)
    - mean_pmax: Mean of P_max (e.g., 1000 W/m²)
    - std_pmax: Standard deviation of P_max (e.g., 100 W/m²)
    - seed: Random seed for reproducibility
    
    Returns:
    - irradiance: numpy array of length 24, representing hourly solar irradiance
    - p_max: the actual peak irradiance drawn for the day
    """
    if seed is not None:
        np.random.seed(seed)
    
    hours = np.arange(24 + 1)  # 0 to 25 
    irradiance = np.zeros_like(hours, dtype=float)
    
    t_mid = (sunrise + sunset) / 2
    daylight_duration = sunset - sunrise

    # Draw P_max from normal distribution
    p_max = np.random.normal(loc=mean_pmax, scale=std_pmax)

    # Apply cosine model during daylight hours
    for h in range(24 + 1):
        
        if sunrise <= h <= sunset:
            val = p_max * np.cos(np.pi * (h - t_mid) / daylight_duration)
            irradiance[h] = max(0, val)  # clip negative values to zero

        #ensuring that the 25th hour is the same as the 0th hour
        if (h == 25): val[h] == val[0] 

    return irradiance, p_max

# Example usage
# irradiance_profile, p_max = generate_daily_solar_profile(seed=0)
# print(f"P_max for the day: {p_max:.2f}")
# print("Hourly irradiance values:")
# print(irradiance_profile)

def generate_nday_solar(sunrise=6, sunset=20, mean_pmax=14, std_pmax=1, days= 1):
    """Generate nday solar production"""

    irradiance_profile = []
    p_max_list = []
    for day in range(days): 
        #print(f"Day = {day}")
        power_gen_day, p_max = generate_daily_solar_profile(sunrise, sunset, mean_pmax, std_pmax, day)
        #print(f"P_max for the day: {p_max:.2f}")
        #print("Hourly irradiance values:")

        if day == 0: 
    #we want 25 values for day 1 
            irradiance_profile.extend(power_gen_day)
        else: #we want less than 25 values for day 2 
            irradiance_profile.extend(power_gen_day[1:])

        p_max_list.append(p_max)
    
    return irradiance_profile, p_max_list

        

## Data Recording Services 

In [ ]:
# results_summary.csv
if not os.path.exists("results_summary.csv"):
    pd.DataFrame(columns=[
        "v_max", "C_max", "p_max", "seed",
        "social_welfare",
    ]).to_csv("results_summary.csv", index=False)

# results_hourly.csv
if not os.path.exists("results_hourly.csv"):
    pd.DataFrame(columns=[
        "v_max", "C_max", "p_max", "seed",
        "hour", "p", "q_s", "q_b", "q_u", "C", "q_d", "s_t"
    ]).to_csv("results_hourly.csv", index=False)


### Full Day simulation: Informed Trader 

In [ ]:

# === Parameters ===
days = 7
T = 24*days 
v_max = 10 #max value for the demand purchased
q_max = 10 #max quantity that demand agent would purchase 
c_u = 5 #utility per unit marginal cost 
q_u_max = 10 #max procurement by the utility
C_init = 1
C_max = 20
q_b_max = 2

# === Simulated solar irradiance ===
sunrise = 6 
sunset = 20
mean_pmax = 20
std_pmax = 0


solar_gen_profile, p_max_list = generate_nday_solar(sunrise, sunset, mean_pmax, std_pmax, days)

T = T+1
#np.random.seed(0)
s_t = np.array(solar_gen_profile)  
#s_t = np.array([6,6,6])
q_u_max = np.ones(T)*q_max #max capacity that the utility can supply 
q_max_arr = np.ones(T)*q_max
v_max_arr = np.ones(T)*v_max

k = 100

reserve_x = np.sqrt(k/5) #number of Energy Tokens in the pool 
reserve_y = k/reserve_x #number of Money Tokens in the pool
p = reserve_y/reserve_x

#Demand/Consumer/Load agent 
demand_agent = ElasticDemand(v_max_arr, q_max_arr, "demand", True )
utility_agent = Supply(c_u, q_u_max, "utility", True)
solar_agent = Supply(0, s_t, "solar", True )
battery_agent = InformedTraderBattery(C_init, C_max,"informed_trader_battery", s_t, v_max, q_max, c_u, q_u_max, q_b_max ,verbose= True)


#AMM 
amm_agent = AMM()
reserve_x,reserve_y = 4, 25

amm_agent.setup_pool(reserve_x, reserve_y)
agent_list = [utility_agent, demand_agent, utility_agent, battery_agent]
agent_list = [demand_agent,utility_agent,solar_agent, battery_agent]

trades_per_period = 20

#Model Agent 
model = Model(T, agent_list, amm_agent, trades_per_period,  s_t)
model.simulate()
    



In [ ]:
# === Analysis of Comprehensive Trading Data ===
# This cell demonstrates how to use the new data tracking system

# Check if model has run successfully
if hasattr(model, 'data_tracker'):
    print("=== COMPREHENSIVE DATA TRACKING ANALYSIS ===\n")
    
    # Get all the detailed trading data
    trading_data = model.get_trading_data()
    
    # print("=== SIMULATION SUMMARY ===")
    # summary_stats = trading_data['summary_statistics']
    # for key, value in summary_stats.items():
    #     print(f"{key.replace('_', ' ').title()}: {value}")
    
    print("\n=== TRADING ROUNDS DATA ===")
    trading_rounds_df = trading_data['trading_rounds']
    print(f"Shape: {trading_rounds_df.shape}")
    if not trading_rounds_df.empty:
        print("\nFirst few trades:")
        print(trading_rounds_df[['period', 'round', 'agent_type', 'trade_type', 
                               'quantity_electricity', 'money_paid', 'price_per_unit']].head())
    
    print("\n=== PERIOD SUMMARY DATA ===")
    period_summary_df = trading_data['period_summary']
    print(f"Shape: {period_summary_df.shape}")
    if not period_summary_df.empty:
        print("\nPeriod summaries:")
        print(period_summary_df[['period', 'total_trades', 'total_electricity_traded', 
                               'avg_price', 'final_amm_price']].head())
    
    print("\n=== AGENT PERFORMANCE ===")
    agent_summary_df = trading_data['agent_summary']
    if not agent_summary_df.empty:
        for agent_type in agent_summary_df['agent_type'].unique():
            agent_data = agent_summary_df[agent_summary_df['agent_type'] == agent_type]
            print(f"\n{agent_type.replace('_', ' ').title()}:")
            print(f"  Total Trades: {len(agent_data)}")
            print(f"  Total Electricity Traded: {agent_data['quantity_electricity'].sum():.2f}")
            print(f"  Total Money Exchanged: {agent_data['money_paid'].sum():.2f}")
            print(f"  Average Price: {agent_data['price_per_unit'].mean():.4f}")
            print(f"  Total Surplus: {agent_data['agent_surplus'].sum():.2f}")
    
    # Export data to CSV files
    # print("\n=== EXPORTING DATA ===")
    # model.export_detailed_results("energy_abm_detailed")
    # print("Detailed results exported to CSV files!")
    
else:
    print("Model needs to be run first. Please execute the simulation cell above.")
    print("\nThe new data tracking system provides:")
    print("- Individual trade records (trading rounds level)")
    print("- Period aggregated summaries") 
    print("- Agent performance metrics")
    print("- Comprehensive surplus and welfare analysis")
    print("- Price impact and volatility tracking")
    print("- Automated CSV export functionality")

In [ ]:
df = model.get_trading_data()["summary_statistics"]


In [ ]:
df = model.get_trading_data()["period_summary"]

len(df)

In [ ]:
df.columns

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(x=df["period"], y=df.solar_total_electricity_sold, mode='lines', name='Solar Dispatch', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df["period"], y=df.informed_trader_battery_total_electricity_sold -df.informed_trader_battery_total_electricity_bought, mode='lines', name='Battery Dispatch', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df["period"], y=df.utility_total_electricity_sold, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=df["period"], y=df.demand_total_electricity_bought, mode='lines', name='Total Demand', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df["period"], y=df.solar_generation, mode='lines', name='Solar Supply', line=dict(color='gold', dash='dash')))

fig.update_layout(
    title="Dispatch and Market Quantities",
    xaxis_title="Time",
    yaxis_title="Quantity"
    
)

fig.show()

In [ ]:

fig = go.Figure()


fig.add_trace(go.Scatter(x=df["period"], y=df.final_amm_price , mode='lines', name='Final AMM Price ', line=dict(color='red')))


fig.update_layout(
    title="AMM spot price ",
    xaxis_title="Time",
    yaxis_title="Spot Price for electricity"
    
)

fig.show()

In [ ]:
df.final_amm_reserve_x.max()

In [ ]:
df2 = model.data_tracker.get_trading_rounds_df()
df2.columns

In [ ]:

fig = go.Figure()


fig.add_trace(go.Scatter(x=df2["timestamp"], y=df2.amm_price_after, mode='lines', name='Final AMM Price ', line=dict(color='red')))


for period_start in range(T-1): 
    mask = (df2["period"] == period_start)
    indices = df2["period"].index[mask]
    min_index = indices.min()
    #print(min_index)
    fig.add_vline(x = df2.iloc[min_index]["timestamp"])

fig.update_layout(
    title="Per Trade AMM price",
    xaxis_title="Time",
    yaxis_title="Spot Price for electricity"
    
)

fig.show()

In [ ]:

fig = go.Figure()


fig.add_trace(go.Scatter(x=df2["timestamp"], y=df2.amm_reserve_x_after, mode='lines', name='AMM X reserves', line=dict(color='orange')))

fig.add_trace(go.Scatter(x=df2["timestamp"], y=df2.amm_reserve_y_after, mode='lines', name='AMM Y reserves', line=dict(color='green')))
# for period_start in range(T-1): 
#     mask = (df2["period"] == period_start)
#     indices = df2["period"].index[mask]
#     min_index = indices.min()
#     #print(min_index)
#     fig.add_vline(x = df2.iloc[min_index]["timestamp"])

fig.update_layout(
    title="Per Trade AMM price",
    xaxis_title="Time",
    yaxis_title="Spot Price for electricity"
    
)

fig.show()

In [ ]:
max_val = df2.amm_reserve_x_after.max()
rows_with_max = df2[df2.amm_reserve_x_after == max_val]
print(f"Number of times max occurs: {len(rows_with_max)}")
rows_with_max

In [ ]:
df2.amm_reserve_y_after.min()

In [ ]:

# === Parameters ===
days = 7
T = 24*days 
v_max = 10 #max value for the demand purchased
q_max = 10 #max quantity that demand agent would purchase 
c_u = 5 #utility per unit marginal cost 
q_u_max = 10 #max procurement by the utility
C_init = 8 #2
C_max = 10 #10
q_b_max = 1 

# === Simulated solar irradiance ===
sunrise = 6 
sunset = 20
mean_pmax = 15
std_pmax = 7.5


solar_gen_profile, p_max_list = generate_nday_solar(sunrise, sunset, mean_pmax, std_pmax, days)

#np.random.seed(0)
s_t = np.array(solar_gen_profile)  
#s_t = np.array([6,6,6])
q_u_max = np.ones(T)*q_max #max capacity that the utility can supply 
q_max_arr = np.ones(T)*q_max
v_max_arr = np.ones(T)*v_max

k = 100

reserve_x = 4 #number of Energy Tokens in the pool 
reserve_y = k/reserve_x #number of Money Tokens in the pool
p = reserve_y/reserve_x


#Demand/Consumer/Load agent 
demand_agent = ElasticDemand(v_max_arr, q_max_arr, "demand", True )
utility_agent = Supply(c_u, q_u_max, "utility", True)
solar_agent = Supply(0, s_t, "solar", True )
battery_agent = OptimalPolicyBattery(v_max,q_max, c_u, 10, C_init, C_init, C_max, s_t, 1, T, q_b_max, "vfi_optimized_battery")


#AMM 
amm_agent = AMM()
reserve_x = np.sqrt(k/5)
reserve_y = k/reserve_x
amm_agent.setup_pool(reserve_x, reserve_y)
agent_list = [utility_agent, demand_agent, solar_agent, battery_agent]
#agent_list = [battery_agent]

trades_per_period = 20

#Model Agent 
model = Model(T, agent_list, amm_agent, trades_per_period,  s_t)
model.simulate()
    



In [ ]:
# === Analysis of Comprehensive Trading Data ===
# This cell demonstrates how to use the new data tracking system

# Check if model has run successfully
if hasattr(model, 'data_tracker'):
    print("=== COMPREHENSIVE DATA TRACKING ANALYSIS ===\n")
    
    # Get all the detailed trading data
    trading_data = model.get_trading_data()
    
    # print("=== SIMULATION SUMMARY ===")
    # summary_stats = trading_data['summary_statistics']
    # for key, value in summary_stats.items():
    #     print(f"{key.replace('_', ' ').title()}: {value}")
    
    print("\n=== TRADING ROUNDS DATA ===")
    trading_rounds_df = trading_data['trading_rounds']
    print(f"Shape: {trading_rounds_df.shape}")
    if not trading_rounds_df.empty:
        print("\nFirst few trades:")
        print(trading_rounds_df[['period', 'round', 'agent_type', 'trade_type', 
                               'quantity_electricity', 'money_paid', 'price_per_unit']].head())
    
    print("\n=== PERIOD SUMMARY DATA ===")
    period_summary_df = trading_data['period_summary']
    print(f"Shape: {period_summary_df.shape}")
    if not period_summary_df.empty:
        print("\nPeriod summaries:")
        print(period_summary_df[['period', 'total_trades', 'total_electricity_traded', 
                               'avg_price', 'final_amm_price']].head())
    
    print("\n=== AGENT PERFORMANCE ===")
    agent_summary_df = trading_data['agent_summary']
    if not agent_summary_df.empty:
        for agent_type in agent_summary_df['agent_type'].unique():
            agent_data = agent_summary_df[agent_summary_df['agent_type'] == agent_type]
            print(f"\n{agent_type.replace('_', ' ').title()}:")
            print(f"  Total Trades: {len(agent_data)}")
            print(f"  Total Electricity Traded: {agent_data['quantity_electricity'].sum():.2f}")
            print(f"  Total Money Exchanged: {agent_data['money_paid'].sum():.2f}")
            print(f"  Average Price: {agent_data['price_per_unit'].mean():.4f}")
            print(f"  Total Surplus: {agent_data['agent_surplus'].sum():.2f}")
    
    # Export data to CSV files
    # print("\n=== EXPORTING DATA ===")
    # model.export_detailed_results("energy_abm_detailed")
    # print("Detailed results exported to CSV files!")
    
else:
    print("Model needs to be run first. Please execute the simulation cell above.")
    print("\nThe new data tracking system provides:")
    print("- Individual trade records (trading rounds level)")
    print("- Period aggregated summaries") 
    print("- Agent performance metrics")
    print("- Comprehensive surplus and welfare analysis")
    print("- Price impact and volatility tracking")
    print("- Automated CSV export functionality")

In [ ]:
df = model.get_trading_data()["period_summary"]
df.columns

In [ ]:

import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(x=df["period"], y=df.solar_total_electricity_sold, mode='lines', name='Solar Dispatch', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df["period"], y=df.vfi_optimized_battery_total_electricity_sold -df.vfi_optimized_battery_total_electricity_bought, mode='lines', name='Battery Dispatch', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df["period"], y=df.utility_total_electricity_sold, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=df["period"], y=df.demand_total_electricity_bought, mode='lines', name='Total Demand', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df["period"], y=df.solar_generation, mode='lines', name='Solar Supply', line=dict(color='gold', dash='dash')))

fig.update_layout(
    title="Dispatch and Market Quantities",
    xaxis_title="Time",
    yaxis_title="Quantity"
    
)

fig.show()

In [ ]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=df["period"], y=df.final_amm_price , mode='lines', name='Final AMM Price ', line=dict(color='red')))


fig.update_layout(
    title="Dispatch and Market Quantities",
    xaxis_title="Time",
    yaxis_title="Spot Price for electricity"
    
)

fig.show()

In [ ]:
abm_sim_df = pd.read_csv("/Users/nalin/Desktop/UChicago/Thesis/abm_sim_summary.csv")


In [ ]:
abm_sim_df.columns

In [ ]:
import pandas as pd
import re
import ast
import plotly.graph_objects as go

# Load your CSV
abm_sim_df = pd.read_csv("/Users/nalin/Desktop/UChicago/Thesis/abm_sim_summary.csv")

# --- Parser Helpers ---

def parse_series_string(series_str):
    float_strings = re.findall(r"[-+]?\d*\.\d+|\d+", series_str)
    return [float(num) for num in float_strings]

def parse(col):
    if isinstance(col, str) and "dtype:" in col:
        return parse_series_string(col)
    elif isinstance(col, str):
        try:
            return ast.literal_eval(col)
        except (ValueError, SyntaxError):
            return parse_series_string(col)
    elif hasattr(col, 'values'):
        return col.values.tolist()
    elif hasattr(col, '__iter__') and not isinstance(col, str):
        return list(col)
    else:
        return [col]

# --- Plotting ---

def plot_summary_row(row, title_suffix=""):
    battery_type = row["battery_type"]

    if battery_type == "informed":
        q_b_list = parse(row['q_b_inf'])
        socs = parse(row['soc_inf'])
    elif battery_type == "optimal":
        q_b_list = parse(row['q_b_vfi'])
        socs = parse(row['soc_vfi'])
    else:
        q_b_list, socs = [], []

    prices = parse(row['prices'])
    q_s_list = parse(row['q_s'])
    q_u_list = parse(row['q_u'])
    q_d_list = parse(row['q_d'])
    time_index = list(range(len(prices)))

    fig_battery = go.Figure()
    fig_battery.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines+markers', name='Battery Dispatch', line=dict(color='red')))
    fig_battery.add_hline(y=0, line_dash="dash", line_color="gray")
    fig_battery.update_layout(
        title=f"Optimal Battery Dispatch{title_suffix}",
        xaxis_title="Time", yaxis_title="Battery Dispatch (q_b)",
        xaxis=dict(range=[0, len(time_index)])
    )

    return fig_battery




In [ ]:
import plotly.graph_objects as go
import ast

def plot_summary_row(row, title_suffix=""):
    # Helper to parse the stringified lists
    def parse(col):
        if isinstance(col, str):
            return ast.literal_eval(col)
        return col  # already a list
    
    battery_type = row["battery_type"]

    if battery_type == "informed":
        q_b_list = parse(row['q_b_inf'])
        socs = parse(row['soc_inf'])
    elif battery_type == "optimal":
        q_b_list = parse(row['q_b_vfi'])
        socs = parse(row['soc_vfi'])
    else:
        q_b_list, socs = [], []

    prices = parse(row['prices'])
    q_s_list = parse(row['q_s'])
    q_u_list = parse(row['q_u'])
    q_d_list = parse(row['q_d'])
    # For solar supply curve, you can use q_s or supply your own s_t if you saved it.

    
    time_index = list(range(len(prices)))

    # 1. Battery Dispatch (q_b)
    fig_battery = go.Figure()
    fig_battery.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines+markers', name=f'Battery Dispatch: {battery_type}', line=dict(color='red')))
    fig_battery.add_hline(y=0, line_dash="dash", line_color="gray")
    fig_battery.update_layout(title=f"Optimal Battery Dispatch {battery_type} {title_suffix}", xaxis_title="Time", yaxis_title="Battery Dispatch (q_b)", 
                            xaxis=dict(range=[0, len(time_index)]))

    # 2. Battery State of Charge (SOC)
    fig_soc = go.Figure()
    fig_soc.add_trace(go.Scatter(x=time_index, y=socs, mode='lines+markers', name='SOC', line=dict(color='green')))
    fig_soc.add_hline(y=socs[0], line_dash="dot", line_color="orange", annotation_text=f"Initial/Final SOC {battery_type}", annotation_position="bottom right")
    fig_soc.update_layout(title=f"Battery State-of-Charge (SOC){battery_type} {title_suffix}", xaxis_title="Time", yaxis_title="SOC",
                          yaxis=dict(range=[0, max(prices)*1.1]))

    # 3. Market Clearing Price
    fig_price = go.Figure()
    fig_price.add_trace(go.Scatter(x=time_index, y=prices, mode='lines+markers', name='Market Price', line=dict(color='blue')))
    fig_price.update_layout(title=f"Market Clearing Price{title_suffix}", xaxis_title="Time", yaxis_title="Price")

    # 4. Dispatch Quantities
    fig_dispatch = go.Figure()
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_s_list, mode='lines', name='Solar Dispatch', line=dict(color='green')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines', name='Battery Dispatch', line=dict(color='red')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_u_list, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_d_list, mode='lines', name='Total Demand', line=dict(color='blue')))
    # Optionally, add Solar Supply if you saved s_t
    if 's_t' in row:
        s_t = parse(row['s_t'])
        fig_dispatch.add_trace(go.Scatter(x=time_index, y=s_t, mode='lines', name='Solar Supply', line=dict(color='orange', dash='dash')))
    
    fig_dispatch.update_layout(
        title=f"Dispatch and Market Quantities{title_suffix}",
        xaxis_title="Time", yaxis_title="Quantity"
    )

    return {
        "battery_dispatch": fig_battery,
        "state_of_charge": fig_soc,
        "market_price": fig_price,
        "dispatch_quantities": fig_dispatch
    }

# Usage example:
# row = summary_df.iloc[0]
# figs = plot_summary_row(row)
# figs['battery_dispatch'].show()
# figs['state_of_charge'].show()
# figs['market_price'].show()
# figs['dispatch_quantities'].show()


In [ ]:
plot_dict = plot_summary_row(abm_sim_df.iloc[902])

In [ ]:
plot_dict["dispatch_quantities"].show()

In [ ]:
plot_dict = plot_summary_row(abm_sim_df.iloc[913])
plot_dict["dispatch_quantities"].show()

In [ ]:
period_data = pd.read_csv("/Users/nalin/Desktop/UChicago/Thesis/abm_period_summary.csv")


In [ ]:
period_data[(period_data["period"] == 0)]["informed_trader_battery_total_electricity_bought"]

In [ ]:
period_data[(period_data["period"] == 0)]["informed_trader_battery_total_electricity_sold"]

In [ ]:
period_data[(period_data["period"] == 0)]["vfi_optimized_battery_total_electricity_bought"]

In [ ]:
new_df = period_data[(period_data["period"] == 0)& (period_data["vfi_optimized_battery_total_electricity_sold"] > 0)][['C_max', 'C_init', 'q_b_max', 'mean_pmax', 'std_pmax', 'days', 'T',
       'sunrise', 'sunset', 'v_max', 'q_max', 'c_u', 'q_u_max', 'beta',
       'trades_per_period', 'battery_type', 'reserve_x_init',
       'reserve_y_init',"vfi_optimized_battery_total_electricity_sold"]].reset_index()

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
new_df

In [ ]:
row = abm_sim_df[
    (abm_sim_df['C_max'] == 20) &
    (abm_sim_df['C_init'] == 16) &
    (abm_sim_df['q_b_max'] == 2) &
    (abm_sim_df['mean_pmax'] == 20) &
    (abm_sim_df['std_pmax'] == 0)
]
row